<a href="https://colab.research.google.com/github/ArneHei/Backend_Mobility/blob/main/Classes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class Shipment:
    registry = {}

    def __init__(self, Shipment_ID, Transport, Department, Pickup_time, Pickup_date, Delivery_time, Delivery_date,
                 Collection_Name, Collection_City, Collection_Address, Collection_Postal_Code, Collection_Country,
                 Delivery_Name, Delivery_City, Delivery_Address, Delivery_Postal_Code, Delivery_Country,
                 Weight, Volume, Ldm, Content, Units, Unit_type, Hazardous, Cost,
                 Finance_Department="", Incoterm="", Customer="", Loading_Instructions="", Customer_Reference="", Additional_Information="", Services=[],
                 min_temperature=-99.0, max_temperature=99.0):
        self.Shipment_ID = Shipment_ID
        self.Transport = Transport
        self.Department = Department
        self.Pickup_time = Pickup_time
        self.Pickup_date = Pickup_date
        self.Delivery_time = Delivery_time
        self.Delivery_date = Delivery_date
        self.Collection_Name = Collection_Name
        self.Collection_City = Collection_City
        self.Collection_Address = Collection_Address
        self.Collection_Postal_Code = Collection_Postal_Code
        self.Collection_Country = Collection_Country
        self.Delivery_Name = Delivery_Name
        self.Delivery_City = Delivery_City
        self.Delivery_Address = Delivery_Address
        self.Delivery_Postal_Code = Delivery_Postal_Code
        self.Delivery_Country = Delivery_Country
        self.Weight = Weight
        self.Volume = Volume
        self.Ldm = Ldm
        self.Content = Content
        self.Units = Units
        self.Unit_type = Unit_type
        self.Hazardous = Hazardous
        self.Cost = Cost
        self.Finance_Department = Finance_Department
        self.Incoterm = Incoterm
        self.Customer = Customer
        self.Loading_Instructions = Loading_Instructions
        self.Customer_Reference = Customer_Reference
        self.Additional_Information = Additional_Information
        self.Services = Services
        self.min_temperature = float(min_temperature)
        self.max_temperature = float(max_temperature)

        Shipment.registry[Shipment_ID] = self

        # Generate stops upon initialization
        self.stops = []
        try:
            pickup_stop = Stop(self, 'P')
            self.stops.append(pickup_stop)
        except ValueError as e:
            print(f"Warning: Could not create pickup stop for shipment {self.Shipment_ID}: {e}")

        try:
            delivery_stop = Stop(self, 'D')
            self.stops.append(delivery_stop)
        except ValueError as e:
            print(f"Warning: Could not create delivery stop for shipment {self.Shipment_ID}: {e}")

    @classmethod
    def get_by_id(cls, shipment_id):
        return cls.registry.get(shipment_id)

    def __repr__(self):
        return f"<Shipment(Shipment_ID='{self.Shipment_ID}', Department='{self.Department}', Transport='{self.Transport}')>"

In [ ]:
class Stop:
    registry = {}

    def __init__(self, shipment_obj, stop_type):
        """
        Initializes a Stop object.

        Args:
            shipment_obj (Shipment): The Shipment object associated with this stop.
            stop_type (str): The type of stop, either 'P' for Pickup or 'D' for Delivery.
        """
        if not isinstance(shipment_obj, Shipment):
            raise TypeError("shipment_obj must be an instance of the Shipment class.")
        if stop_type not in ['P', 'D']:
            raise ValueError("stop_type must be 'P' or 'D'.")

        self.shipment = shipment_obj
        self.Type = stop_type
        self.ID = f"{shipment_obj.Shipment_ID}_{stop_type}" # Reverted ID format

        if self.ID in Stop.registry:
            raise ValueError(f"Stop with ID '{self.ID}' already exists.")
        Stop.registry[self.ID] = self

        # Inherit additional attributes from the Shipment object
        self.Transport = shipment_obj.Transport
        self.Weight = shipment_obj.Weight
        self.Volume = shipment_obj.Volume
        self.Ldm = shipment_obj.Ldm
        self.Content = shipment_obj.Content # Content is already present in the shipment, so inherit it
        self.Units = shipment_obj.Units
        self.Unit_type = shipment_obj.Unit_type
        self.Hazardous = shipment_obj.Hazardous

        # Add new attributes
        self.Additional_Information = shipment_obj.Additional_Information
        self.Services = shipment_obj.Services

        if self.Type == 'P': # Shortened from 'Pickup'
            self.Time = shipment_obj.Pickup_time
            self.Date = shipment_obj.Pickup_date
            self.Address = shipment_obj.Collection_Address
            self.City = shipment_obj.Collection_City
            self.Name = shipment_obj.Collection_Name
            self.Postal_Code = shipment_obj.Collection_Postal_Code
            self.Instructions = shipment_obj.Loading_Instructions # Specific for Pickup
        elif self.Type == 'D': # Shortened from 'Delivery'
            self.Time = shipment_obj.Delivery_time
            self.Date = shipment_obj.Delivery_date
            self.Address = shipment_obj.Delivery_Address
            self.City = shipment_obj.Delivery_City
            self.Name = shipment_obj.Delivery_Name
            self.Postal_Code = shipment_obj.Delivery_Postal_Code
            self.Instructions = shipment_obj.Customer_Reference # Specific for Delivery

    @classmethod
    def get_by_id(cls, stop_id):
        """
        Retrieves a Stop object by its ID.
        """
        return cls.registry.get(stop_id)

    def __repr__(self):
        return f"<Stop(ID='{self.ID}', Type='{self.Type}', City='{self.City}', Date='{self.Date}')>"

In [ ]:
class Transport:
    registry = {}

    def __init__(self, Transport_ID, Department, shipments_list, Pickup_date, Delivery_date, Weight, Volume, Ldm, Cost):
        if Transport_ID in Transport.registry:
            raise ValueError(f"Transport with ID '{Transport_ID}' already exists.")
        self.Transport_ID = Transport_ID
        self.Department = Department
        self.Shipments = shipments_list
        self.Pickup_date = Pickup_date
        self.Delivery_date = Delivery_date
        self.Weight = Weight
        self.Volume = Volume
        self.Ldm = Ldm
        self.Cost = Cost

        # Initialize these attributes with default values
        self.Status = 'Planning'
        self.Vehicle = ''
        self.Haulier = ''
        self.Driver = ''
        self.Trailer = ''
        self.Haulier_cost = 0.0
        self.Sale = False
        self.Sale_cost = 0.0

        # Collect all Stop objects from the associated Shipments
        self.Stops = []

        Transport.registry[self.Transport_ID] = self

    @classmethod
    def get_by_id(cls, Transport_ID):
        return cls.registry.get(Transport_ID)

    def __repr__(self):
        return f"<Transport(ID='{self.Transport_ID}', Department='{self.Department}', Shipments={self.Shipments}, Weight={self.Weight}, Volume={self.Volume}, Ldm={self.Ldm}, status={self.Status}, vehicle={self.Vehicle}, Haulier={self.Haulier}, Driver={self.Driver}, Trailer={self.Trailer}, Cost={self.Cost})>"

In [ ]:
class Truck:
    registry = {}

    def __init__(self, License_plate, Driver, Trailer, Phone, Haulier, Time, Date, Location, Transport, Department, Pay_type, Pay_amount, Last_transport=''):
        if License_plate in Truck.registry:
            raise ValueError(f"Truck with license plate '{License_plate}' already exists.")
        self.License_plate = License_plate
        self.Driver = Driver
        self.Trailer = Trailer
        self.Phone = Phone
        self.Haulier = Haulier
        self.Time = Time
        self.Date = Date
        self.Location = Location
        self.Transport = Transport
        self.Department = Department
        self.Pay_type = Pay_type
        self.Pay_amount = Pay_amount
        self.Last_transport = Last_transport

        Truck.registry[self.License_plate] = self

    @classmethod
    def get_by_id(cls, license_plate):
        return cls.registry.get(license_plate)

    def __repr__(self):
        return f"<Truck(License_plate='{self.License_plate}', Driver='{self.Driver}', Haulier='{self.Haulier}', Transport='{self.Transport}', Trailer='{self.Trailer}', Location='{self.Location}')>"


In [ ]:
class Trailer:
    registry = {}

    def __init__(self, License_plate, Open_pool, Type, Sub_type, Department='', Transport=None, Start_date=None, End_date=None):
        if License_plate in Trailer.registry:
            raise ValueError(f"Trailer with license plate '{License_plate}' already exists.")
        self.License_plate = License_plate
        self.Open_pool = Open_pool
        self.Transport = Transport
        self.Start_date = Start_date
        self.End_date = End_date
        self.Transport = Transport
        self.Start_date = Start_date
        self.End_date = End_date
        self.Department = Department
        self.Type = Type
        self.Sub_type = Sub_type
        self.Transport = None  # Initialize as blank/None
        self.Start_date = None  # Initialize as blank/None
        self.End_date = None  # Initialize as blank/None

        Trailer.registry[self.License_plate] = self

    @classmethod
    def get_by_id(cls, license_plate):
        return cls.registry.get(license_plate)

    def __repr__(self):
        return f"<Trailer(License_plate='{self.License_plate}', Department='{self.Department}', Type='{self.Type}', Sub type = '{self.Sub_type}')>"